In [6]:
import pandas as pd
from pyspark.sql import *

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
chats = SQLContext(sc).read.csv("chats.csv").rdd
chats = chats.map(lambda x:(pd.to_numeric(x[0]),pd.to_numeric(x[1]),x[2],x[3])).cache()

In [11]:
#Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con 
#el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario 
#(user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter 
#“?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.

mensajes_con_pregunta = chats.filter(lambda x:"?" in x[3]).\
        map(lambda x:(x[1],(x[2],x[3].count("?"))))#(id_usuario,(nickname,cantidad_de_preguntas))
preguntas_totales = mensajes_con_pregunta.reduceByKey(lambda x,y:(x[0],x[1]+y[1]))
preguntas_totales.reduce(lambda x,y:x if x[1][1]>y[1][1] else y)

(19, ('mastodonte', 34))